In [ ]:
import pandas as pd
import tensorflow as tf
from google.colab import drive
import pickle

drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
piTrain=pickle.load(open('/content/drive/MyDrive/DL_Project3/DS_5_train_input_prefixList','rb'))
nwTrain=pickle.load(open('/content/drive/MyDrive/DL_Project3/DS_5_train_input_nextWord','rb'))

train_full=pickle.load(open('/content/drive/MyDrive/DL_Project3/DS_5_train_input','rb'))

In [ ]:
def max_seq_length(data):
  max_len=0
  unique_letters=[]
  for text in data:
    # print(text)
    text_letters=text
    unique_letters+=text_letters
    if len(text_letters)>max_len:
      max_len=len(text_letters)
      if max_len==48:
        print(text)
  vocab_list=list(set(unique_letters))
  vocab_size=len(vocab_list)
  return max_len,vocab_list,vocab_size
max_train_ip_len,train_vocab_ip_list,train_vocab_ip_size=max_seq_length(piTrain)
max_train_ip_len,train_vocab_ip_list,train_vocab_ip_size=max_seq_length(train_full)


['a', 'k', 'b', 'd', 'a', 'j', 'b', 'f', 'a', 'f', 'a', 'k', 'a', 'j', 'c', 'f', 'b', 'f', 'b', 'e', 'b', 'd', 'a', 'e', 'a', 'd', 'c', 'e', 'a', 'g', 'a', 'f', 'a', 'e', 'a', 'g', 'a', 'j', 'c', 'g', 'c', 'f', 'b', 'e', 'a', 'k', 'c', 'e']


In [ ]:
piTrain_str=[]
for i in piTrain:
  piTrain_str.append([" ".join(i)])

text_pairs=[]
text_data_total=[]
for i in range(len(piTrain_str)):
  text_pairs.append((piTrain_str[i], nwTrain[i]))
  text_data_total.append(piTrain_str[i][0]+" "+
                         nwTrain[i])


In [ ]:
num_val_samples = int(0.15 * len(text_data_total))
num_train_samples =  len(text_data_total) - 2 * num_val_samples
train_data_total = text_pairs[:num_train_samples]
val_data_total = text_pairs[num_train_samples:num_train_samples + num_val_samples]

test_pairs = text_pairs[num_train_samples + num_val_samples:]


In [ ]:
from tensorflow.keras.layers import TextVectorization

sequence_length = 50
vocab_size = 12
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length
)

train_input_texts = [val[0] for val in text_pairs]
text_vectorization.adapt(train_input_texts)

def format_dataset(prefix, nextword):
    prefix = text_vectorization(prefix)
    nextword = text_vectorization(nextword)
    return (prefix,nextword)

In [ ]:
batch_size=32
def make_dataset(pairs):
    input_texts, target_texts = zip(*pairs)
    input_texts = list(input_texts)
    target_texts = list(target_texts)
    dataset = tf.data.Dataset.from_tensor_slices((input_texts, target_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()


dataset1 = make_dataset(train_data_total)
dataset_validation = make_dataset(val_data_total)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import LayerNormalization, Dense, Layer, Attention


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)

        self.attention_3 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=4*embed_dim)

        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.layernorm_4 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)

        attention_output_3 = self.attention_3(
            attention_output_2,
            attention_output_2
        )
        attention_output_3 = self.layernorm_3(
            attention_output_3 + attention_output_2)

        proj_output = self.dense_proj(attention_output_3)
        return self.layernorm_4(attention_output_3 + proj_output)



In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [ ]:
import numpy as np
from tensorflow.keras import layers
import keras
embed_dim = 20
latent_dim = 256
num_heads = 2

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop",metrics=["accuracy"])
idx_token = dict(enumerate(text_vectorization.get_vocabulary()))

In [ ]:
model.fit(dataset1, 
            epochs= 250,
            validation_data=dataset_validation
            )

Epoch 1/250
110/110 [==============================] - 14s 91ms/step - loss: 0.1676 - accuracy: 0.9556 - val_loss: 0.0826 - val_accuracy: 0.9716
Epoch 2/250
110/110 [==============================] - 8s 74ms/step - loss: 0.0810 - accuracy: 0.9698 - val_loss: 0.0799 - val_accuracy: 0.9716
Epoch 3/250
110/110 [==============================] - 8s 76ms/step - loss: 0.0799 - accuracy: 0.9699 - val_loss: 0.0797 - val_accuracy: 0.9716
Epoch 4/250
110/110 [==============================] - 9s 84ms/step - loss: 0.0797 - accuracy: 0.9699 - val_loss: 0.0796 - val_accuracy: 0.9716
Epoch 5/250
110/110 [==============================] - 7s 66ms/step - loss: 0.0796 - accuracy: 0.9701 - val_loss: 0.0795 - val_accuracy: 0.9716
Epoch 6/250
110/110 [==============================] - 10s 91ms/step - loss: 0.0795 - accuracy: 0.9701 - val_loss: 0.0795 - val_accuracy: 0.9716
Epoch 7/250
110/110 [==============================] - 8s 76ms/step - loss: 0.0795 - accuracy: 0.9702 - val_loss: 0.0795 - val_accurac

In [ ]:
model.save('/content/drive/MyDrive/DL/DL Project 3/vandy/project3_model.h5')

In [ ]:
def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)


def nextword_generation(sentence,temperature):
  generate_length=1
  length=len(sentence[0].split(" "))
  tokenized_sentence = text_vectorization([sentence])
  predictions = model(tokenized_sentence)
  next_token = sample_next(predictions[0, 0, :],temperature)
  sampled_token = idx_token[next_token]
  return sampled_token


test_data_ip_str = [pair[0] for pair in test_pairs]
test_target_texts = [pair[1] for pair in test_pairs]
decoded_nextwords=[]
for input_sentence in test_data_ip_str:
  sampled_token=nextword_generation(input_sentence,temperature=1)
  decoded_nextwords.append(sampled_token)

In [ ]:
def findAccuracy(test_input,test_decoded_sentences):
    count = 0
    total_count=len(test_input)
    for i in range(total_count):
        if test_input[i] == test_decoded_sentences[i]:
            count += 1
    print(count,total_count)
    accuracy=count/total_count*100
    return accuracy


In [ ]:
accuracy=findAccuracy(test_target_texts,decoded_nextwords) 
print("accuracy",accuracy)

213 750
accuracy 28.4


**test data output**

In [ ]:
test_data=pickle.load(open('/content/drive/MyDrive/DL_Project3/DS_6_test_input_prefixList','rb'))

In [ ]:
test_data_str=[]
for i in test_data:
  test_data_str.append([" ".join(i)])

decoded_nextwords_final=[]
for input_sentence in test_data_str:
  sampled_token=nextword_generation(input_sentence,temperature=1)
  decoded_nextwords_final.append(sampled_token)

pickle.dump(decoded_nextwords_final, open(
  "/content/drive/MyDrive/DL/DL Project 3/Untitled Folder/ActualTestDataOutputList.pkl",'wb'))